## Setup & Config

In [ ]:
# --- Environment & Imports ---
import os, sys, glob, json
from pathlib import Path

# Make sure both the repo root and a potential 'modules/' folder are importable
ROOT = Path.cwd()
if (ROOT / "modules").exists():
    sys.path.insert(0, str(ROOT / "modules"))
sys.path.insert(0, str(ROOT))

# Basic utilities
import matplotlib.pyplot as plt

# YAML config loader (fallback if modules.config_utils not found)
try:
    from modules.config_utils import load_config  # project helper if available
except Exception:
    import yaml
    def load_config(path):
        with open(path, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)

# Helper to auto-detect a config file if not set
DEFAULT_CONFIG_CANDIDATES = ["config.yaml", "config.yml", "configs/config.yaml"]
for _cand in DEFAULT_CONFIG_CANDIDATES:
    if Path(_cand).exists():
        CONFIG_PATH = _cand
        break
else:
    # Put a placeholder if nothing found
    CONFIG_PATH = "config.yaml"  # adjust if needed
print(f"Using CONFIG_PATH = {CONFIG_PATH!r}")
CONFIG = load_config(CONFIG_PATH)
CONFIG

## Run drought processing

In [ ]:
# Imports with fallback (monolithic repo vs. modular layout)
try:
    from modules.drought_module import process_drought, plot_ndmi_raster
except Exception:
    from drought_module import process_drought, plot_ndmi_raster

# Execute drought pipeline (uses CONFIG sections like hazards.drought, input paths, output_dir, etc.)
drought_raster_path = process_drought(CONFIG)
print("Drought raster (if produced):", drought_raster_path)

# Optionally plot NDMI raster if a path was returned and the function expects a path
try:
    if drought_raster_path:
        plot_ndmi_raster(drought_raster_path, CONFIG.get("aoi"))
except Exception as e:
    print("[WARN] Plotting NDMI raster failed:", e)

In [ ]:
# --- Browse outputs ---
out_dir = Path(CONFIG.get("output_dir", "output"))
out_dir.mkdir(parents=True, exist_ok=True)
maps = sorted(list(out_dir.rglob("*.png"))) + sorted(list(out_dir.rglob("*.jpg")))
rasters = sorted(list(out_dir.rglob("*.tif"))) + sorted(list(out_dir.rglob("*.tiff")))
vectors = sorted(list(out_dir.rglob("*.gpkg"))) + sorted(list(out_dir.rglob("*.geojson")))

print(f"Found {len(maps)} map images, {len(rasters)} rasters, {len(vectors)} vectors under {out_dir}")

# Show the latest map if available
if maps:
    from PIL import Image
    display(Image.open(maps[-1]))
else:
    print("No map images found yet.")